<a href="https://colab.research.google.com/github/pctablet505/BalanceFL/blob/master/fedrated_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pctablet505/BalanceFL.git balancedfl_repo

Cloning into 'balancedfl_repo'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 139 (delta 73), reused 125 (delta 66), pack-reused 0 (from 0)
Receiving objects: 100% (139/139), 683.12 KiB | 3.06 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [2]:
%cd /content/balancedfl_repo/
!git pull

/content/balancedfl_repo
Already up to date.


In [3]:
# 2. Get training command for your DeepFed model
!python CIFAR10/simple_train.py --config CIFAR10/config/deepfed.yaml --model deepfed

Training deepfed with config CIFAR10/config/deepfed.yaml
Experiment: simple_exp
Config loaded:
- Dataset: DeepFed
- Model: DeepFed
- Classes: 8
- Output: ./runs_simple/simple_exp
- Device: cuda
- Mode: Federated (5 clients, 100 rounds)

TRAINING STARTED
For federated training, run the original scripts:
python train_ours.py --cfg CIFAR10/config/deepfed.yaml --exp_name simple_exp
python train_fedavg.py --cfg CIFAR10/config/deepfed.yaml --exp_name simple_exp
python train_fedprox.py --cfg CIFAR10/config/deepfed.yaml --exp_name simple_exp
Use the commands above to start actual training!

Next steps:
1. Check the suggested command above
2. Results will be saved in: ./runs_simple/simple_exp
3. Use tensorboard to view training: tensorboard --logdir ./runs_simple/simple_exp


In [9]:
!python CIFAR10/train_ours.py --cfg CIFAR10/config/deepfed.yaml --exp_name simple_exp

2025-10-06 03:43:58.551325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759722238.582077    2578 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759722238.591456    2578 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759722238.613929    2578 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759722238.613958    2578 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759722238.613966    2578 computation_placer.cc:177] computation placer alr

In [ ]:
!head -n 80 CIFAR10/train_ours.py

In [ ]:
!sed -i '/imb_ratio:/a\  tao_ratio: 0.0' CIFAR10/config/deepfed.yaml

In [ ]:
!sed -i '/exp_name:/a\  work_dir: ./runs_simple/simple_exp' CIFAR10/config/deepfed.yaml

In [ ]:
!sed -i '/metainfo:/a\  exp_name: simple_exp' CIFAR10/config/deepfed.yaml

In [ ]:
!cat CIFAR10/config/deepfed.yaml

fl_opt:
  rounds: 100
  num_clients: 5
  frac: 1.0         # the fraction of clients in each FL round
  local_ep: 10      # local epoch
  local_bs: 32      # local batch size
  aggregation: fedavg  # fedavg, fedavgm, fedbn, fedfs

criterions:
  def_file: ./loss/KDLoss.py    # for cross entropy
  loss_params: {Temp: 2, lamda: 0, loss_cls: ce, loss_kd: kl}

networks:
  feat_model:
    def_file: DeepFed
    params: 
      input_shape: [26, 1]
      gru_hidden_size: 128
    optim_params: {lr: 0.001, momentum: 0.9, weight_decay: 0.0001}
    feat_dim: 512
    fix: false
  classifier:
    def_file: DotProductClassifier
    params: {num_classes: 8, l2_norm: false, bias: true, scale: 1}   
    optim_params: {lr: 0.001, momentum: 0.9, weight_decay: 0.0001} 
    fix: false
    
dataset:
  name: DeepFed
  num_classes: 8
  shot_few: 0   # if shot_few>0, (iid + non-iid) mixed mode; else, non-mixed mode
  non_iidness: 1
  imb_ratio: 1.0  # balanced data for intrusion detection

training:
  epochs: 10

In [1]:
!python3 -m pip install ai-edge-litert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 MB 9.3 MB/s eta 0:00:00


In [3]:
from ai_edge_litert.interpreter import Interpreter
